In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pii-detection-removal-from-educational-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F66653%2F7500999%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240408%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240408T150849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D457b98efd6a9947da951d572d3f84f3589f5693931715b0d8848192a91b83d918e8d822924f1b8400454fb5e0c94a537f96fb191d614214d64ffc36000fdd3d25fd24e1c01491ca2e4e6620a577e0378b588c119ba2b5ab4ea7ea994c9b15362b608b764fb95bd923bfbef0dfebfe3ed3d65152051b13e9902c50954d0ff41fdd6166678315c3f87c79b5da7f8377c3977a9d78fec10d9d3b2162e986f9a4ebf4c9ab7e8e46f49657115d87b7a651e2d44265eb54ba1fa93f0682bf81aef65f8535615050740022ffae93f5931772203d9b1f1482d36c77b3a976c4416981cc53104849cdea2ba4bb3e19962bca8a8ec6e1cd8dace58f64261e17c1adca2f05d,pii-external-dataset:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4332496%2F7466758%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240408%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240408T150849Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc8c4f9e98ea8b186d27166f7f1a4802f483b975987222bbeaac89aac399c2227e21549a74b4712742f541fe9bbae3d2a24bfc3b937a6447ec550a68159e8ab47d297af144a6726a16bf548fd8af1bb45792f5433bd4bfdeb000b67a03f31d0220cf1718d8bc4efbcef8094f499fc520e0146d68633c272082ffdf17a3890aa0674129668c5cec0241e310abc28317a4fae7698976b905139f628478390fdda19694618a369c2bbe2fcd37b9b14904d58aae46701f7300e90df4751d8050f58d6619ebcccfbaafc9472bf11cff852cd847fca5112af1ea6eb58a5b2aa40a4b0144205c1487a28477f99ad8fdb3844ff3a561f7ca9ea0812f1a0aeaa3335049647'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Configuration

In [ ]:
class CONFIG:
    """
    > General Options
    """
    # global seed
    seed = 42
    # number of samples to use for testing purposes
    # if None, we use the whole training dataset
    samples_testing = None
    # flag to indicate whether to use the external training dataset
    # or just to use the original data
    use_external_train_data = True
    # whether to run the algorithm on the training set and do subsequent validation
    # with 6.8k rows, this takes almost 50 minutes to run
    run_on_train_data = True

    """
    > Analyzer Options
    """
    # score threshold for patterns
    id_pattern_score = 0.8
    address_pattern_score = 0.8
    email_pattern_score = 0.8
    url_pattern_score = 0.8

# Import Libraries

### Install presidio

In [ ]:

!pip install -U -q presidio_analyzer --no-index --find-links=file:///kaggle/input/presidio-wheels/presidio

### Import  necessary libraries

In [ ]:
import json
import pandas as pd

from presidio_analyzer import AnalyzerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from tqdm import tqdm
from typing import List
import random
import pprint
import re
import gc
from ast import literal_eval
from collections import Counter

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import fbeta_score, classification_report, confusion_matrix

from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult
from presidio_analyzer.recognizer_registry import RecognizerRegistry
from presidio_analyzer.nlp_engine import NlpEngine, SpacyNlpEngine, NlpArtifacts
from presidio_analyzer.context_aware_enhancers import LemmaContextAwareEnhancer

from dateutil import parser

In [ ]:
random.seed(42)

# Define Metric (F1 Beta Score)

In [ ]:
# Big thanks to
# https://www.kaggle.com/code/amedprof/pii-evaluation-metric?scriptVersionId=160040455
def pii_fbeta_score(pred_df, gt_df,beta=5):
    """
    Parameters:
    - pred_df (DataFrame): DataFrame containing predicted PII labels.
    - gt_df (DataFrame): DataFrame containing ground truth PII labels.
    - beta (float): The beta parameter for the F-beta score, controlling the trade-off between precision and recall.

    Returns:
    - float: Micro F-beta score.
    """
    df = pred_df.merge(gt_df,how='outer',on=['document',"token"],suffixes=('_pred','_gt'))

    df['cm'] = ""

    df.loc[df.label_gt.isna(),'cm'] = "FP"


    df.loc[df.label_pred.isna(),'cm'] = "FN"
    df.loc[(df.label_gt.notna()) & (df.label_gt!=df.label_pred),'cm'] = "FN"

    df.loc[(df.label_pred.notna()) & (df.label_gt.notna()) & (df.label_gt==df.label_pred),'cm'] = "TP"

    FP = (df['cm']=="FP").sum()
    FN = (df['cm']=="FN").sum()
    TP = (df['cm']=="TP").sum()

    s_micro = (1+(beta**2))*TP/(((1+(beta**2))*TP) + ((beta**2)*FN) + FP)

    return s_micro

# Import Datasets

## Import Original Data

In [ ]:
train_df = json.load(open("/kaggle/input/pii-detection-removal-from-educational-data/train.json"))
print(f"len(train_df):{len(train_df)}, train_df[0].keys(): {list(train_df[0].keys())}")
print("-"*50)

test_df = json.load(open('/kaggle/input/pii-detection-removal-from-educational-data/test.json'))

## Load External Data (if needed)

In [ ]:
if CONFIG.use_external_train_data:
    # Convert the "stringified lists" in the columns to proper Python lists
    df_train_external = pd.read_csv('/kaggle/input/pii-external-dataset/pii_dataset.csv', converters={
        'tokens': literal_eval,
        'labels': literal_eval,
        'trailing_whitespace': literal_eval
    })
    df_train_external.rename(columns={'text': 'full_text'}, inplace=True)
    # convert to format similar to how we load in the original data
    df_train_external = df_train_external.to_dict('records')
    train_df.extend(df_train_external)

## Sample Data (if needed)

In [ ]:
if CONFIG.samples_testing != None:
    train_df = random.sample(train_df, CONFIG.samples_testing)

In [ ]:
print(f"train_df length:", len(train_df))

labels = set()
label_counts = {}
for i in range(len(train_df)):
    labels.update(train_df[i]['labels'])
    for label in train_df[i]['labels']:
        if label in label_counts:
            label_counts[label] += 1
        else:
            label_counts[label] = 1

print(f"labels: {labels}")
print('-'*25)
print(f"label_counts: {label_counts}")

## Helper Methods

In [ ]:
def is_valid_date(text):
    try:
        # Attempt to parse the text as a date
        parsed_date = parser.parse(text)
        return True
    except:
        return False

def tokens2index(row):
    tokens  = row['tokens']
    start_ind = []
    end_ind = []
    prev_ind = 0
    for tok in tokens:
        start = prev_ind + row['full_text'][prev_ind:].index(tok)
        end = start+len(tok)
        start_ind.append(start)
        end_ind.append(end)
        prev_ind = end
    return start_ind, end_ind

# binary search
def find_or_next_larger(arr, target):
    left, right = 0, len(arr) - 1

    while left <= right:
        mid = (left + right) // 2

        if arr[mid] == target:
            return mid
        elif arr[mid] < target:
            left = mid + 1
        else:
            right = mid - 1
    return left

def count_trailing_whitespaces(word):
    return len(word) - len(word.rstrip())

## Create Analyzer

For ease of code, we encapsulate the analyzer code in a class.

In [ ]:
class MyAnalyzer:

    def __init__(self):
        ## Initialize the analyzer
        configuration = {
            "nlp_engine_name": "spacy",
            "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}],
        }

        # Create NLP engine based on configuration
        provider = NlpEngineProvider(nlp_configuration=configuration)
        nlp_engine = provider.create_engine()

        # create ID recognizer
        id_regex = r'([A-Za-z]{2}[.?]:)?\d{12,12}'
        id_pattern = Pattern(name="id", regex=id_regex, score = CONFIG.id_pattern_score)
        id_recognizer = PatternRecognizer(supported_entity="ID_CUSTOM", patterns = [id_pattern])

        # create address recognizer
        address_regex = r'\b\d+\s+\w+(\s+\w+)*\s+((st(\.)?)|(ave(\.)?)|(rd(\.)?)|(blvd(\.)?)|(ln(\.)?)|(ct(\.)?)|(dr(\.)?))\b'
        address_pattern = Pattern(name="address", regex=address_regex, score = CONFIG.address_pattern_score)
        address_recognizer = PatternRecognizer(supported_entity="ADDRESS_CUSTOM", patterns = [address_pattern])

        # create email recognizer
        email_regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
        email_pattern = Pattern(name="email address", regex=email_regex, score = CONFIG.email_pattern_score)
        email_recognizer = PatternRecognizer(supported_entity="EMAIL_CUSTOM", patterns = [email_pattern])

        # create url recognizer
        url_regex = r'https?://\S+|www\.\S+'
#         url_regex = r'https?:\/\/[a-zA-Z1-9.\?\=\&]+'
        url_pattern = Pattern(name="url", regex=url_regex, score=CONFIG.url_pattern_score)
        url_recognizer = PatternRecognizer(supported_entity="URL_CUSTOM", patterns = [url_pattern])

        registry = RecognizerRegistry()
        registry.load_predefined_recognizers()
        registry.add_recognizer(id_recognizer)
        registry.add_recognizer(address_recognizer)
        registry.add_recognizer(email_recognizer)
        registry.add_recognizer(url_recognizer)

        # Pass the created NLP engine and supported_languages to the AnalyzerEngine
        self.analyzer = AnalyzerEngine(
            nlp_engine=nlp_engine,
            supported_languages=["en"],
            registry=registry
        )

        ## Initialize the black list
        self.black_list = ["wikipedia", "coursera", ".pdf", ".PDF", "article",
                           ".png", ".gov", ".work", ".ai", ".firm", ".arts",
                           ".store", ".rec", ".biz", ".travel" ]


    def predict_tokens(self, df_: list) -> pd.DataFrame:
        """Predict the tokens that have PII in the dataframe."""

        # list of all predictions for each label
        PHONE_NUM, NAME_STUDENT, URL_PERSONAL, EMAIL, STREET_ADDRESS, ID_NUM, USERNAME = [],[],[],[],[],[], []

        preds = []

        # Find the starting and ending positions of each word after segmentation
        for i in tqdm(range(len(df_)), desc="Processing tokens2index"):
            start, end = tokens2index(df_[i])
            df_[i]['start'] = start
            df_[i]['end'] = end

        for i, d in tqdm(enumerate(df_), total=len(df_), desc="Analyzing entities"):
            #results:[type: PERSON, start: 22, end: 37, score: 0.85]
            results = self.analyzer.analyze(text=d['full_text'],
                                   entities=[
                                             #"PHONE_NUMBER",
                                             "PERSON",
                                             "URL_CUSTOM", #"IP_ADDRESS", #"URL",
                                             "EMAIL_ADDRESS", "EMAIL_CUSTOM",
                                             "ADDRESS_CUSTOM",
                                             "US_SSN", "US_ITIN", "US_PASSPORT", "US_BANK_NUMBER",
                                             "USERNAME"],
                                   language='en',
        #                            score_threshold=0.2,
                                    )
            pre_preds = []

            # Traverse each entity found,
            # r: [type: PERSON, start: 22, end: 37, score: 0.85]
            for r in results:
                # That is, the sth word is the beginning of an entity
                s = find_or_next_larger(d['start'], r.start)# d['start'][s] = r.start

                end = r.end # entity end point
                # find words in text
                word = d['full_text'][r.start:r.end]
                end = end - count_trailing_whitespaces(word)
                temp_preds = [s]

                try:
                    while d['end'][s+1] <= end:
                        temp_preds.append(s+1)
                        s +=1
                except:
                    pass

                tmp = False

                if r.entity_type == 'USERNAME':
                    label = 'USERNAME'
                    USERNAME.append(d['full_text'][r.start:r.end])

        #         if r.entity_type == 'PHONE_NUMBER':
        #             if is_valid_date(word):
        #                 continue
        #             label =  'PHONE_NUM'
        #             PHONE_NUM.append(d['full_text'][r.start:r.end])

                if r.entity_type == 'PERSON':
                    label = 'NAME_STUDENT'
                    NAME_STUDENT.append(d['full_text'][r.start:r.end])

                if r.entity_type == 'ADDRESS_CUSTOM':
                    label = 'STREET_ADDRESS'
                    STREET_ADDRESS.append(d['full_text'][r.start:r.end])

                if r.entity_type == 'ID_CUSTOM' or \
                    r.entity_type == 'US_SSN' or r.entity_type == 'US_ITIN' or r.entity_type == 'US_PASSPORT' or r.entity_type == 'US_BANK_NUMBER':

                    label = 'ID_NUM'
                    ID_NUM.append(d['full_text'][r.start:r.end])

                if r.entity_type == 'EMAIL_ADDRESS' or r.entity_type == 'EMAIL_CUSTOM':
                    label = 'EMAIL'
                    EMAIL.append(d['full_text'][r.start:r.end])

                if r.entity_type == 'URL_CUSTOM':# or r.entity_type == 'IP_ADDRESS' or "http" in word:
                    for w in self.black_list:
                        if w in word:
                            tmp = True
                            break

                    label = 'URL_PERSONAL'
                    URL_PERSONAL.append(d['full_text'][r.start:r.end])

                if tmp:
                    continue

                for p in temp_preds:
                    if len(pre_preds) > 0:
                        """
                        When starting a new r, pre_preds[-1]['rlabel']
                        is still the r.entity_type of the previous entity
                        At this time, it may not be equal to the
                        r.entity_type of this entity.

                        In other words,
                        the first equal sign is still in the same entity.
                        """
                        if pre_preds[-1]['rlabel'] == r.entity_type and (p - pre_preds[-1]['token']==1):
                            label_f = "I-"+label
                        else:
                            label_f = "B-"+label
                    else:
                        label_f = "B-"+label

                    # pre_preds contains the output that we want
                    pre_preds.append(({
                        "document": d['document'],
                        "token": p,
                        "label": label_f,
                        "rlabel": r.entity_type,
                    }))

            # After traversing this data, summarize all found entities
            # and extend the preds for this document into the aggregate preds
            preds.extend(pre_preds)

        preds_df = pd.DataFrame(preds).iloc[:,:-1].reset_index()
        return preds_df

# Predict Train Set

In [ ]:
analyzer = MyAnalyzer()

In [ ]:
if CONFIG.run_on_train_data:
    train_preds = analyzer.predict_tokens(train_df)

# Evaluate Performance on Training Set

## Generate Corresponding DataFrame for "True" Answers

In [ ]:
if CONFIG.run_on_train_data:
    train_act_records = []
    count = 0
    for entry in train_df:
        for idx, (token, label) in enumerate(zip(entry["tokens"], entry["labels"])):
            if label != 'O':
                train_act_records.append({
                    'row_id': count,
                    'document': entry["document"],
                    'token': idx,
                    'label': label,
                })
                count += 1

    train_act = pd.DataFrame.from_records(train_act_records)

In [ ]:
# Check that we haven't missed out any true values when making the "train_act_records"
if CONFIG.run_on_train_data:
    assert len(train_act) == sum(label_counts.values()) - label_counts['O'], \
        'mismatch between number of true labels in label_counts and train_act_records'

In [ ]:
def get_pred_act_lists_for_dfs(preds: pd.DataFrame, act: pd.DataFrame):
    document_idx_list = [(ex["document"], len(ex["tokens"])) for ex in train_df]

    preds_list = []
    act_list = []

    for document, len_tokens in tqdm(document_idx_list, total=len(document_idx_list)):
        preds_doc = preds[preds["document"] == document].sort_values(by="token")
        act_doc = act[act["document"] == document].sort_values(by="token")

        # We do a "merge" (like in mergesort) to combine the results from the preds and
        # actual values
        preds_idx = 0
        act_idx = 0
        preds_list_sub = []
        act_list_sub = []
        for i in range(len_tokens):
            preds_head, act_head = None, None
            if preds_idx < len(preds_doc):
                preds_head = preds_doc.iloc[preds_idx]
            if act_idx < len(act_doc):
                act_head = act_doc.iloc[act_idx]

            if act_head is not None and act_head["token"] == i:
                act_list_sub.append(act_head["label"])
                act_idx += 1
            else:
                act_list_sub.append('O')

            if preds_head is not None and preds_head["token"] == i:
                preds_list_sub.append(preds_head["label"])
                preds_idx += 1
            else:
                preds_list_sub.append('O')

        preds_list.extend(preds_list_sub)
        act_list.extend(act_list_sub)

    return preds_list, act_list


In [ ]:
if CONFIG.run_on_train_data:
    print("PII micro F-beta score:", pii_fbeta_score(train_preds, train_act, beta = 5))

In [ ]:
if CONFIG.run_on_train_data:
    train_preds_list, train_act_list = get_pred_act_lists_for_dfs(train_preds, train_act)

In [ ]:
# make sure that we processed the actual train list properly
if CONFIG.run_on_train_data:
    assert dict(Counter(train_act_list)) == label_counts, 'mismatch between label counts in label_counts and train_act_list'

In [ ]:
if CONFIG.run_on_train_data:
    print("-"*25)
    print("Counter for predicted labels:")
    print("-"*25)
    pprint.pprint(dict(Counter(train_preds_list)))
    print()

    print("-"*25)
    print("Counter for actual labels:")
    print("-"*25)
    pprint.pprint(dict(Counter(train_act_list)))
    print()

## Classification Report on Training Set

In [ ]:
if CONFIG.run_on_train_data:
    print(classification_report(train_preds_list, train_act_list, digits=4))

## F Beta Score on Training Set

In [ ]:
# if CONFIG.run_on_train_data:
#     print("Micro F1 Beta Score:", score(train_preds_list, train_act_list))
#     print("Macro F1 Beta Score:", macro_score(train_preds_list, train_act_list))

In [ ]:
if CONFIG.run_on_train_data:
    del train_preds_list, train_act_list, train_preds, train_act
    gc.collect()

# Predict Test Set

In [ ]:
test_preds = analyzer.predict_tokens(test_df)
test_preds.head()

# Submission

In [ ]:
submission = pd.DataFrame(test_preds)
submission.columns = ['row_id','document', 'token', 'label']
submission.to_csv('submission.csv', index = False)
submission.head()